In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [2]:
rows = 5      #记得行数和列数这里要同步改
columns = 5
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 

gridworld.show()
value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [3]:
def SARSA(gridworld:GridWorld_v2.GridWorld_v2,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是最基础的SARSA算法

    Parameters:
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))  # 初始化状态价值函数为0
    action_value = np.zeros((rows * columns, 5))  # 初始化动作价值函数Q表为0
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))]  # 随机初始化策略，使用独热编码表示
    epsilon = 0.5  # 初始化epsilon值为0.5，用于epsilon-greedy策略
    for episode in range(num_episodes):  # 循环迭代指定次数
        #清除输出，可以更好的展示策略
        # time.sleep(0.2)
        # clear_output(wait=True)
        
        print("episode",f"{episode}/{num_episodes}")  # 打印当前迭代次数
        if(epsilon > final_epsilon) :  # 如果当前epsilon大于最终epsilon值
            epsilon -= 0.001  # 则逐渐减小epsilon值
        else:
            epsilon = final_epsilon  # 否则保持epsilon为最终值

        # p1是目标方向的概率，p0是另外四个方向的概率
        p1 = 1-epsilon * (4/5)  # 计算选择最优动作的概率
        p0 = epsilon/5  # 计算选择其他动作的概率
        d = {1:p1, 0:p0}  # 创建概率字典，用于向量化操作
        # policy_epsilon是policy取epsilon-greedy的概率决策
        print("p1",p1,"p0",p0)  # 打印当前的概率值
        policy_epsilon = np.vectorize(d.get)(policy)  # 将策略转换为epsilon-greedy概率形式

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]  # 初始化访问计数器
        
        initState=10  # 设置初始状态为10
        initAction=random.randint(0,4)  # 随机选择初始动作

        if trajectorySteps==-1:  # 如果轨迹步数为-1
            stop_when_reach_target = True  # 则设置到达目标时停止
        Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                  action=initAction, 
                                                  policy=policy_epsilon, 
                                                  steps=trajectorySteps, 
                                                  stop_when_reach_target=True)  # 获取轨迹
        Trajectory.append((17,4,1,17,4))  # 添加一个自循环状态，确保最后的奖励被更新，因为目标位置（对号)在17,为了目标位置的奖励被更新
        print("trajectorySteps",len(Trajectory))  # 打印轨迹长度
        

        
        # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
        steps = len(Trajectory) - 1  # 计算实际步数
        for k in range(steps,-1,-1):  # 从后向前遍历轨迹
            #State，Action，Reward，NextState，NextAction
            tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]  # 解包当前步骤的信息
            cnt[tmpstate] += 1  # 增加该状态的访问计数
            #SARSA，根据公式更新action_value
            TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState][nextAction])  # 计算TD误差
            action_value[tmpstate][tmpaction] -= learning_rate * TD_error  # 使用TD误差更新Q值

        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  # 策略改进：选择Q值最大的动作作为新策略
        policy_epsilon = np.vectorize(d.get)(policy)  # 将新策略转换为epsilon-greedy形式
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5))  # 打印状态访问次数矩阵

        state_value = np.sum(policy_epsilon * action_value,axis=1)  # 计算状态价值函数
        mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean()  # 计算平均状态价值
        
        gridworld.showPolicy(policy)  # 显示当前策略
        print(np.round(state_value,decimals=4).reshape(5,5))  # 打印状态价值矩阵（保留4位小数）
        print("mean_state_value", mean_state_value)  # 打印平均状态价值

    return action_value  # 返回最终的动作价值函数

In [4]:
action_value = SARSA(gridworld)

episode 0/600
p1 0.6008 p0 0.0998
trajectorySteps 28
[[0 0 1 4 2]
 [0 0 1 2 2]
 [1 3 2 7 1]
 [0 0 2 0 0]
 [0 0 0 0 0]]
⬆️⬆️⬆️⬆️⬆️
⬆️⏫️⏫️⬇️➡️
⬆️⬆️⏬➡️➡️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
[[ 0.      0.     -0.001  -0.     -0.    ]
 [ 0.      0.     -0.001  -0.     -0.    ]
 [-0.     -0.001   0.0006 -0.     -0.    ]
 [ 0.      0.     -0.0004  0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]]
mean_state_value -0.0001119686794262584
episode 1/600
p1 0.6015999999999999 p0 0.0996
trajectorySteps 301
[[57 50 23 17 30]
 [13  9  2  7 39]
 [ 3  3  1  9 34]
 [ 0  0  2  1  1]
 [ 0  0  0  0  0]]
⬇️🔄➡️🔄🔄
⬇️⏪⏪⬅️⬅️
⬇️⬇️⏬⬆️⬇️
⬆️⏫️✅⏩️➡️
⬆️⏫️⬆️⬆️⬆️
[[-0.0041 -0.0085 -0.0035 -0.001  -0.0016]
 [-0.0022 -0.002  -0.001  -0.     -0.0026]
 [-0.     -0.002   0.0012 -0.002  -0.0023]
 [ 0.      0.     -0.0008 -0.     -0.    ]
 [ 0.      0.      0.      0.      0.    ]]
mean_state_value -0.0012931055122094902
episode 2/600
p1 0.6024 p0 0.0994
trajectorySteps 106
[[ 5  0  0  0  0]
 [12  2  0  0  0]
 [36  8  0  0  0]
 [3

In [5]:
action_value

array([[-1.75940987e-01, -1.33428025e-02, -1.60646127e-02,
        -1.72018799e-01, -1.43798542e-02],
       [-9.34188536e-02, -8.71276044e-03, -7.88449097e-01,
        -1.01330272e-02, -9.25555985e-03],
       [-1.08105106e-01, -6.38234468e-03, -6.86710957e-01,
        -8.14787984e-03, -6.93737446e-03],
       [-1.93489047e-01, -1.07985718e-02, -4.58246183e-03,
        -9.69142804e-03, -4.79661582e-03],
       [-1.53431259e-01, -1.83861099e-01, -2.11672102e-03,
        -9.13132028e-03, -1.09986155e-02],
       [-9.76594606e-03, -1.11466679e+00, -1.01572901e-02,
        -7.14231799e-02, -1.64002291e-02],
       [-1.96835532e-03, -1.49180590e-01, -3.13522660e-03,
        -2.05577237e-03, -1.49177500e-01],
       [-2.62906471e-04, -2.17432064e-04, -8.94575081e-02,
        -1.48974166e-01, -1.09472530e-01],
       [-2.01283525e-03, -2.33529306e-03, -1.89388537e-03,
        -5.36382377e-01, -3.13140945e-03],
       [-4.01687292e-03, -1.51570935e-01, -4.58113817e-03,
        -4.63698268e-03